Reference : https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['hindienglish-corpora', 'zinditraduction']


In [ ]:
lines=pd.read_csv("../input/zinditraduction/Train (13).csv")

In [ ]:
lines.head()

,ID,Yoruba,English
0,ID_AAJEQLCz,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú.,A Disaster Relief Committee was formed to organize the long-term relief efforts.
1,ID_AASNedba,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákùnrin Solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti Anna ìyàwó rẹ̀ ń bọ̀ láti ìrìn-àjò tí wọ́n lọ lórílẹ̀-èdè míì.","Brother Solovyev was arrested on the evening of May 22, 2018, at a railway station, as he was arriving home from a trip abroad with his wife, Anna."
2,ID_AAeQrhMq,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,ID_AAxlMgPP,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo̩lé, Senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù, wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú Colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ H ló̩jó̩bò̩.","With Egypt, Morocco and Tunisia out of the World Cup, Senegal, the only African nation left, will take on Columbia in their last group H game on Thursday."
4,ID_ABKuMKSx,"Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney General of the Federation), Justice Abubakar Malami ti pàsẹ fún ilé-isẹ́ ọ̀tẹlẹ̀múyẹ́ (Department of State Services, DSS) láti jáwọ́ lórí ẹ̀sun Omoyẹle Sowore ní kíákíá.","The Attorney General of the Federation, Justice Abubakar Malami has ordered the Department of State Services (DSS) to hands off the trial of Omoyele Sowore."


In [ ]:
pd.isnull(lines).sum()

ID         0
Yoruba     0
English    0
dtype: int64

In [ ]:
lines=lines[~pd.isnull(lines['Yoruba'])]

In [ ]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [ ]:
#lines=lines.sample(n=25000,random_state=42)
lines.shape

(10054, 3)

In [ ]:
# Lowercase all characters
lines['Yoruba']=lines['Yoruba'].apply(lambda x: x.lower())
lines['English']=lines['English'].apply(lambda x: x.lower())

In [ ]:
# Remove quotes
lines['Yoruba']=lines['Yoruba'].apply(lambda x: re.sub("'", '', x))
lines['English']=lines['English'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['Yoruba']=lines['Yoruba'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['English']=lines['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['Yoruba']=lines['Yoruba'].apply(lambda x: x.translate(remove_digits))
lines['English']=lines['English'].apply(lambda x: x.translate(remove_digits))

lines['English'] = lines['English'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['Yoruba']=lines['Yoruba'].apply(lambda x: x.strip())
lines['English']=lines['English'].apply(lambda x: x.strip())
lines['Yoruba']=lines['Yoruba'].apply(lambda x: re.sub(" +", " ", x))
lines['English']=lines['English'].apply(lambda x: re.sub(" +", " ", x))


In [ ]:
# Add start and end tokens to target sequences
lines['English'] = lines['English'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.head()

,ID,Yoruba,English
0,ID_AAJEQLCz,a ṣètò ìgbìmọ̀ tó ń ṣètò ìrànwọ́ nígbà àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú,START_ a disaster relief committee was formed to organize the longterm relief efforts _END
1,ID_AASNedba,ìrọ̀lẹ́ may ni wọ́n fàṣẹ ọba mú arákùnrin solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti anna ìyàwó rẹ̀ ń bọ̀ láti ìrìnàjò tí wọ́n lọ lórílẹ̀èdè míì,START_ brother solovyev was arrested on the evening of may at a railway station as he was arriving home from a trip abroad with his wife anna _END
2,ID_AAeQrhMq,iléeṣẹ́ creative commons náà,START_ creative commons the organization _END
3,ID_AAxlMgPP,pè̩lú egypt morocco àti tunisia tí wó̩n ti lo̩lé senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ h ló̩jó̩bò̩,START_ with egypt morocco and tunisia out of the world cup senegal the only african nation left will take on columbia in their last group h game on thursday _END
4,ID_ABKuMKSx,adájọ́ àgbà lórílẹ̀ èdè náíjíríà attorney general of the federation justice abubakar malami ti pàsẹ fún iléisẹ́ ọ̀tẹlẹ̀múyẹ́ department of state services dss láti jáwọ́ lórí ẹ̀sun omoyẹle sowore ní kíákíá,START_ the attorney general of the federation justice abubakar malami has ordered the department of state services dss to hands off the trial of omoyele sowore _END


In [ ]:
### Get English and Yoruba Vocabulary
all_Yoruba_words=set()
for eng in lines['Yoruba']:
    for word in eng.split():
        if word not in all_Yoruba_words:
            all_Yoruba_words.add(word)

all_Eng_words=set()
for hin in lines['English']:
    for word in hin.split():
        if word not in all_Eng_words:
            all_Eng_words.add(word)

In [ ]:
len(all_Eng_words)

16319

In [ ]:
len(all_Yoruba_words)

15790

In [ ]:
lines['length_Yoruba_sentence']=lines['Yoruba'].apply(lambda x:len(x.split(" ")))
lines['length_English_sentence']=lines['English'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()

,ID,Yoruba,English,length_Yoruba_sentence,length_English_sentence
0,ID_AAJEQLCz,a ṣètò ìgbìmọ̀ tó ń ṣètò ìrànwọ́ nígbà àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú,START_ a disaster relief committee was formed to organize the longterm relief efforts _END,27,14
1,ID_AASNedba,ìrọ̀lẹ́ may ni wọ́n fàṣẹ ọba mú arákùnrin solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti anna ìyàwó rẹ̀ ń bọ̀ láti ìrìnàjò tí wọ́n lọ lórílẹ̀èdè míì,START_ brother solovyev was arrested on the evening of may at a railway station as he was arriving home from a trip abroad with his wife anna _END,29,28
2,ID_AAeQrhMq,iléeṣẹ́ creative commons náà,START_ creative commons the organization _END,4,6
3,ID_AAxlMgPP,pè̩lú egypt morocco àti tunisia tí wó̩n ti lo̩lé senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ h ló̩jó̩bò̩,START_ with egypt morocco and tunisia out of the world cup senegal the only african nation left will take on columbia in their last group h game on thursday _END,28,30
4,ID_ABKuMKSx,adájọ́ àgbà lórílẹ̀ èdè náíjíríà attorney general of the federation justice abubakar malami ti pàsẹ fún iléisẹ́ ọ̀tẹlẹ̀múyẹ́ department of state services dss láti jáwọ́ lórí ẹ̀sun omoyẹle sowore ní kíákíá,START_ the attorney general of the federation justice abubakar malami has ordered the department of state services dss to hands off the trial of omoyele sowore _END,31,27


In [ ]:
lines[lines['length_Yoruba_sentence']>90].shape

(19, 5)

In [ ]:
lines[lines['length_English_sentence']>90].shape

(7, 5)

In [ ]:
#lines=lines[lines['length_Yoruba_sentence']<=20]
#lines=lines[lines['length_English_sentence']<=20]

In [ ]:
lines.shape

(10054, 5)

In [ ]:
print("maximum length of Yoruba Sentence ",max(lines['length_Yoruba_sentence']))
print("maximum length of English Sentence ",max(lines['length_English_sentence']))

maximum length of Yoruba Sentence  218
maximum length of English Sentence  160


In [ ]:
max_length_src=max(lines['length_Yoruba_sentence'])
max_length_tar=max(lines['length_English_sentence'])

In [ ]:
input_words = sorted(list(all_Yoruba_words))
target_words = sorted(list(all_Eng_words))
num_encoder_tokens = len(all_Yoruba_words)
num_decoder_tokens = len(all_Eng_words)
num_encoder_tokens, num_decoder_tokens

(15790, 16319)

In [ ]:
num_decoder_tokens += 1 #for zero padding


In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,ID,Yoruba,English,length_Yoruba_sentence,length_English_sentence
3653,ID_SgHlxEUX,mínísítà ti adelé akọ̀wé àgbà fún ilé ìwòsàn alábọ́dé sojú rẹ̀ iwot ndaeyo dúpẹ́ lọ́wọ́ àwọn òbí fún ìtọ́jú ọmọ wọn nígbà tí ó sì ń tẹnumọ pé ẹ̀bùn láti ọ̀dọ̀ ọlọ́run ni ọmọ,START_ the minister who was represented by the acting executive secretary of primary health care centre dr iwot ndaeyo also tasked mothers on early training of their children just as she emphasised that children are gifts from god _END,34,39
9048,ID_ujKtKygz,“wá jẹun” “n ò jẹ” ó fọgọ́rùnún òkèlè tọ́ ọbẹ̀ wò,START_ “come join me at my meal” “thank you but no” still he eats a hundred mouthfuls just to taste the stew _END,11,23
4761,ID_YaZZBLBL,ọmọrí odó pani lọ́tọ̀ ká tó wí pé ká kùn ún lóògùn,START_ a pestle is a lethal weapon in itself let alone after rubbing poison on it _END,12,17
891,ID_EZwhUnkR,ó fẹ́rẹ̀ẹ́ jẹ́ pé gbogbo iṣẹ́ abẹ inú ẹ̀dọ̀ títí kan iṣẹ́ abẹ tá a fi ń pààrọ̀ ẹ̀dọ̀ là ń ṣe láìsí pé wọ́n fa ẹ̀jẹ̀ síni lára,START_ the vast majority of our liver surgeries even transplants are performed entirely without the use of blood _END,29,19
4071,ID_UnrHdrBJ,“bá mi mádìẹ” kì í fi orúnkún bó,START_ “help me catch a chicken” does not scrape his knees _END,8,12
9920,ID_zUIYVBnL,wọ́n tún ti fọwọ́ sí i pé kó máa ṣèbẹ̀wò sọ́dọ̀ dennis ní ọgbà ẹ̀wọ̀n,START_ approval has also been granted for her to visit him at the prison _END,15,15
4039,ID_UiBZHVSc,tó dára bí i ti màmá ẹ,START_ that is as good as your mom _END,7,9
821,ID_EJDLglZL,ní lọ́wọ́lọ́wọ́ báyìí àwa ẹlẹ́rìí jèhófà ti mú bíbélì ìtumọ̀ ayé tuntun jáde lódindi tàbí lápá kan ní èdè mọ́kàndínlọ́gọ́sànán,START_ to date jehovah’s witnesses have made the new world translation available in whole or in part in languages _END,20,20
9809,ID_yvwpcgOY,obìnrinín pẹ́ lọ́jà ó fìgbójú wọlé,START_ a woman tarried too long at the market and returns home with a brazen face _END,6,17
5363,ID_bgidxYYC,olóyè kalẹ̀jayé ní ó wá jẹ́ ohun ìyàlẹ́nu pe ìyókùn nínú ilẹ̀ náà ni àwọn ọmọ asùnta ti wá sọ ibẹ̀ di ibùgbé,START_ chief kalejaiye explained that he was surprised that the rest of the land was occupied by hoodlums with construction of shanties on the land _END,23,26


### Split the data into train and test

In [ ]:
X, y = lines['Yoruba'], lines['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state=42)
X_train.shape, X_test.shape

((9048,), (1006,))

In [ ]:
test=pd.read_csv('../input/zinditraduction/Test (10).csv')
len(test)

6816

### Let us save this data

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [ ]:
latent_dim=300

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    4737000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    4896000     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
model.load_weights('../input/models/nmt_weights.h5')

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Yoruba sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Yoruba sentence: lára wọn ni nǹkan bí ẹgbẹ̀rún lọ́nà ọgọ́jọ àwọn ará wa àti ẹgbẹ̀rún lọ́nà ọgọ́rùnún méjì ó dín mẹ́ta àwọn akẹ́kọ̀ọ́ bíbélì
Actual English Translation:  that number includes nearly of our brothers and sisters as well as over bible students 
Predicted English Translation:  as a result number of our brothers will be rele


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Yoruba sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Yoruba sentence: a kì í fi ogun dán ẹ̀ṣọ́ wò
Actual English Translation:  one does not tease a warrior by saying there is a war or an invasion 
Predicted English Translation:  one does not find a while on a game hand and only


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Yoruba sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Yoruba sentence: ní april ilé ẹjọ́ ìlú surgut pàṣẹ pé kí wọ́n tú arákùnrin yevgeniy fedin àti arákùnrin sergey loginov sílẹ̀ ní àtìmọ́lé
Actual English Translation:  on april the surgut city court ordered the release of brothers yevgeniy fedin and sergey loginov from pretrial detention 
Predicted English Translation:  on april the surgut city court ordered the rel


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Yoruba sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Yoruba sentence: ìbẹ̀rẹ̀ òṣì bí ọmọ ọlọ́rọ̀ là ń rí
Actual English Translation:  at the beginning of ones penury one seems like the child of most prosperous parents 
Predicted English Translation:  at the beginning of ones penury one seems like


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Yoruba sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Yoruba sentence: ìmìtìtì ilẹ̀ ṣọṣẹ́ ní gúúsù california
Actual English Translation:  earthquakes rattle southern california 
Predicted English Translation:  earthquakes rattle southern california 


## ***Submission***

In [ ]:
test=pd.read_csv('../input/zinditraduction/Test (10).csv')

In [ ]:
test.head(5)

,ID,Yoruba
0,ID_AAAitMaH,"Nínú ìpè kan lẹ́yìn ìgbà náà, wọ́n sọ fún aṣojú iléeṣẹ́ BlaBlaCar pé ètò náà ti yí padà, pé àwọn á máa ṣe ìfọ̀rọ̀wọ́rọ̀ pẹ̀lú àwọn oníbàáràa wọn dípò wọn."
1,ID_AAKKdQwr,Nítorí kò sí nǹkan tí ọkùnrin ò lè ṣe láì nááni nígbà tí nǹkan tó tàsé bá ṣẹlẹ̀.
2,ID_ABgAyEOp,Bí i kó pariwo. Kí ó kígbe mọ́ ẹ?
3,ID_ACFgfKQs,"Tí ó ń lé e lọ sọ́nà etí odò Akókurà, tí ó bẹ̀ẹ́ pé kí ó padà, tí ó ń bẹ́ẹ̀ kí ó tó di wí pé ó kọjá odò náà."
4,ID_ACNPmlhf,Èṣúńiyì mọ̀ iṣẹ́ rẹ̀ dunjú. Màmá tirí bí ó ṣe yanjú irú ìṣòro bẹ́ẹ̀ mẹ́ta ní abúlé .…


In [ ]:
test['English']=""

In [ ]:
X_sub=test['Yoruba']
y_sub=test['English']

In [ ]:
for eng in test['Yoruba']:
    for word in eng.split():
        if word not in all_Yoruba_words:
            all_Yoruba_words.add(word)


In [ ]:
input_words = sorted(list(all_Yoruba_words))

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])

In [ ]:
sub_gen = generate_batch(X_sub, y_sub, batch_size = 1)

In [ ]:
l=[]
for k in range(len(test)):
    (input_seq, actual_output), _ = next(sub_gen)
    decoded_sentence = decode_sequence(input_seq)
    #print('Input Yoruba sentence:', X_sub[k:k+1].values[0])
    #print('Predicted English Translation:', decoded_sentence[:-4])
    l.append(decoded_sentence)
    
    
    
    

In [ ]:
len(test),len(l)

(6816, 6816)

In [ ]:
test['English']=l

In [ ]:
test.head(10)

,ID,Yoruba,English
0,ID_AAAitMaH,"Nínú ìpè kan lẹ́yìn ìgbà náà, wọ́n sọ fún aṣojú iléeṣẹ́ BlaBlaCar pé ètò náà ti yí padà, pé àwọn á máa ṣe ìfọ̀rọ̀wọ́rọ̀ pẹ̀lú àwọn oníbàáràa wọn dípò wọn.",local experiences rogelio jolongbayan regional experience
1,ID_AAKKdQwr,Nítorí kò sí nǹkan tí ọkùnrin ò lè ṣe láì nááni nígbà tí nǹkan tó tàsé bá ṣẹlẹ̀.,in a game interview with global voices and focus the
2,ID_ABgAyEOp,Bí i kó pariwo. Kí ó kígbe mọ́ ẹ?,the squirrel itself for the senior in the wise person
3,ID_ACFgfKQs,"Tí ó ń lé e lọ sọ́nà etí odò Akókurà, tí ó bẹ̀ẹ́ pé kí ó padà, tí ó ń bẹ́ẹ̀ kí ó tó di wí pé ó kọjá odò náà.",mr toriola disclosed that when he was not made enough
4,ID_ACNPmlhf,Èṣúńiyì mọ̀ iṣẹ́ rẹ̀ dunjú. Màmá tirí bí ó ṣe yanjú irú ìṣòro bẹ́ẹ̀ mẹ́ta ní abúlé .…,for the open process i was a blessing on one as a coach
5,ID_ACqxiSuP,Làbákẹ́ wo ọkùnrin náà. Ojú rẹ̀ ti gán-án-ní nọ́mbà ọkọ̀ náà.,nadal with a great feel of the streets with a fight
6,ID_ACyrfZeN,"Ohun tí a kọ sí ojú pátákó: ""Gbé Ìgbésẹ̀ kí o sì kọ àjọ̀dún Òyìnbó"" àti ""Ṣe ìgbélárugẹ àṣà ìbílẹ̀, kọ àjọ̀dún Òyìnbó"".",lagos is a motion of the lagos area dr principles of
7,ID_ADPgGOCq,Bákan náà ni Omarah gbóríyìn fún iléeṣẹ́ eré orí ìtàgé Nollywood ti Nàìjíríà.,for you to have a look from technical explanation and
8,ID_AEJQFohp,Ìrò olóòtú ìwé ìròyìn Punch pe ìhùwàsíi Buhari gẹ́gẹ́ bíi ti apàṣẹ-wàá àti pé ìwà náà lè fa rògbòdìyàn-an ti òfin:,but in a second day of the game looking at the saturday’s
9,ID_AEPshcAq,"Àwọn elétò náà sọ pé ó ""nírú ẹ̀yà"" tí àwọn olugbọ́ àwọn ń fẹ́.",mr seen is the idea of the first time in making the


In [ ]:
test[['ID','English']].to_csv('submission.csv',index=False)